In [0]:
# Tutorial: https://blog.zhaytam.com/2018/08/15/implement-neural-network-backpropagation/
import numpy

In [0]:
class Constants:
  
  SIGMOID = "sigmoid"

In [0]:
class Layer:
  
  # total no. of inputs to the layer
  tot_inputs = None
  # an integer
  # total neurons in the layer
  tot_neurons = None
  # 2d matrix of floats; size = tot_inputs x tot_neurons
  weights = None
  # string; activation function for each neuron in the layer
  act_func = None
  # 1d array of floats; each element is the bias of a neuron; size = no. of neurons
  biases = None
  # 1d array of floats -
  # for input_layer, each element is a feature of a sample or data_point; 
  # for, hidden_layer, each element is the output of a neuron in the previous layer
  inputs = None
  # 1d array; each element is the output of a neuron
  outputs = None
  # error that flows to other layers from current layer during back propagation
  error_out = None
  

In [0]:
class Layer(Layer):
  
  def __init__(self, tot_inputs, tot_neurons, act_func=Constants.SIGMOID):
    self.tot_inputs = tot_inputs
    self.tot_neurons = tot_neurons
    self.act_func = act_func
    # generate random floats in range [0, 1)
    self.weights = numpy.random.rand(self.tot_inputs, self.tot_neurons)
    # generate random numbers in range [0, 1)
    self.biases = numpy.random.rand(self.tot_neurons)

In [0]:
class Layer(Layer):
  
  def apply_act_func(self, x):
    if(self.act_func == Constants.SIGMOID):
      return 1/(1+numpy.exp(-x))

In [0]:
class Layer(Layer):
  
  # returns derivative of activation function
  def apply_act_der(self, x):
    if(self.act_func == Constants.SIGMOID):
      return x*(1-x)

In [0]:
class Layer(Layer):
  
  def feed_forward(self, inputs):
    self.inputs = inputs
    before_act_func = numpy.dot(inputs, self.weights) + self.biases
    self.outputs = self.apply_act_func(before_act_func)

In [0]:
class NeuralNetwork:
  
  # 1d array; each element is of type Layer
  layers = None
  # 2d array; each element is a sample or data_point
  train_x = None
  # 1d array; each element is a label for respective sample in train_x
  train_y = None
  # integer; Total number of epochs.
  tot_epochs = None
  # learning rate during weight updation
  learning_rate = None
  # 1d array; each element is output of neuron in output layer after feed forward
  outputs = None
  

In [0]:
class NeuralNetwork(NeuralNetwork):
  
  def __init__(self, train_x, train_y, tot_epochs, learning_rate):
    self.train_x = train_x
    self.train_y = numpy.reshape(train_y, (len(train_y), 1))
    self.tot_epochs = tot_epochs
    self.learning_rate = learning_rate
    self.layers = []
    self.tot_inputs_to_new_layer = train_x.shape[1]

In [0]:
class NeuralNetwork(NeuralNetwork):
  
  def add_layer(self, tot_neurons, act_func):
    layer = Layer(self.tot_inputs_to_new_layer, tot_neurons, act_func)
    self.layers.append(layer)
    self.tot_inputs_to_new_layer = tot_neurons
    

In [0]:
class NeuralNetwork(NeuralNetwork):
  
  # data: 1d array; a training sample; each element is a feature
  def feed_forward(self, data):
    for layer in self.layers:
      layer.feed_forward(data)
      data = layer.outputs
    self.outputs = data
    

In [0]:
class NeuralNetwork(NeuralNetwork):
  
  # assumption: feed_forward is completed
  def back_prop(self, sample, ground_truth):
    output_layer_ind = len(self.layers) - 1
    cur_layer_ind = output_layer_ind
    while(cur_layer_ind >= 0):
      cur_layer = self.layers[cur_layer_ind]
      if(cur_layer_ind == output_layer_ind):
        # error_in = incoming error = derivative of Mean Squared Error
        prediction = cur_layer.outputs
        error_in = ground_truth - prediction
      else:
        # not the output_layer; a hidden layer.
        next_layer = self.layers[cur_layer_ind+1]
        # error_in = incoming error = weighted error that floats from the next_layer to current_layer
        error_in = numpy.dot(next_layer.weights, next_layer.error_out)
      # error_out = outcoming error = error that flows to other (left layers) layers from current_layer
      cur_layer.error_out = error_in * cur_layer.apply_act_der(cur_layer.outputs)
      cur_layer_ind = cur_layer_ind - 1
    # update weights
    cur_layer_ind = 0
    while(cur_layer_ind <= output_layer_ind):
      cur_layer = self.layers[cur_layer_ind]
      if(cur_layer_ind == 0):
        inp_to_cur_layer = sample
      else:
        # not the 1st hidden layer.
        prev_layer = self.layers[cur_layer_ind-1]
        inp_to_cur_layer = prev_layer.outputs
      # we need a 2d matrix; weights is a 2d matrix
      inp_to_cur_layer = numpy.atleast_2d(inp_to_cur_layer)
      # error that's used for updating weights
      error_upd = cur_layer.error_out * inp_to_cur_layer.T
      cur_layer.weights = cur_layer.weights + (self.learning_rate * error_upd)
      cur_layer_ind = cur_layer_ind + 1

In [0]:
class NeuralNetwork(NeuralNetwork):
  
  def train(self):
    cur_epoch = 0
    while(cur_epoch < self.tot_epochs):
      cur_sample_ind = 0
      while(cur_sample_ind < len(self.train_x)):
        cur_sample = self.train_x[cur_sample_ind]
        cur_label = self.train_y[cur_sample_ind]
        self.feed_forward(cur_sample)
        self.back_prop(cur_sample, cur_label)
        cur_sample_ind = cur_sample_ind + 1
      # compute error on whole train_x i.e all samples for logging
      self.feed_forward(self.train_x)
      last_layer = self.layers[-1]
      mse = numpy.mean(numpy.square(self.train_y - last_layer.outputs))
      print("epoch: " + str(cur_epoch+1) + ", mse: " + str(mse))
      cur_epoch = cur_epoch + 1

In [0]:
class Main:
  
  @staticmethod
  def main():
    train_x = numpy.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    train_y = numpy.array([0, 1, 1, 0])
    tot_epochs = 9
    learning_rate = 0.3
    nn = NeuralNetwork(train_x, train_y, tot_epochs, learning_rate)
    nn.add_layer(3, Constants.SIGMOID)
    nn.add_layer(2, Constants.SIGMOID)
    nn.train()

In [180]:
Main.main()

epoch: 1, mse: 0.39494288853401827
epoch: 2, mse: 0.389534617646196
epoch: 3, mse: 0.383873257365011
epoch: 4, mse: 0.3779755426385345
epoch: 5, mse: 0.37186492964758555
epoch: 6, mse: 0.3655716541453815
epoch: 7, mse: 0.3591323576072299
epoch: 8, mse: 0.352589253388238
epoch: 9, mse: 0.3459888720520628
